In [ ]:

from google.colab import drive

drive.mount("/content/drive")

path = "/content/drive/MyDrive/NLP Datasets/IMDB Dataset.csv"


Mounted at /content/drive


# Libaries and Dependencies

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
import nltk
import numpy as np


In [ ]:
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#Load Data

In [ ]:
data = pd.DataFrame(pd.read_csv(r"/content/drive/MyDrive/NLP Datasets/IMDB Dataset.csv"))

#Pre-Processing

In [ ]:
def text_preprocessing(text):
    lemmatizer = WordNetLemmatizer()
    tokenized_documents = []  # List to store tokenized words for each document

    stopwords_set = set(stopwords.words("english"))

    for corpus in text:
        corpus = re.sub("[^a-zA-Z]", " ", corpus)  # Remove non-alphabetic characters
        corpus = re.sub(r"\s+", " ", corpus)       # Remove extra whitespaces
        corpus = corpus.lower()                    # Convert to lowercase
        words = corpus.split()                     # Tokenize the document into words

        lem_words = []
        for word in words:
            if word not in stopwords_set:
                lemmatized = lemmatizer.lemmatize(word)
                lem_words.append(lemmatized)
        tokenized_documents.append(lem_words)  # Append the tokenized words for this document to the list

    return tokenized_documents

In [ ]:
tokenized_words = text_preprocessing(data["review"])

In [ ]:
tw = tokenized_words
print(len(tw))

50000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
labels = [0 if sentiment == "positive" else 1 for sentiment in data["sentiment"]]


In [ ]:
# 1s
# print(len(tokenized_words))
# 91005

###Example

Only list (i.e) all words in data

In [ ]:
# ['one', 'reviewer', 'mentioned', 'watching', 'oz', 'episode', 'hooked', 'right', 'exactly', 'happened', 'br', 'br',
#  'first', 'thing', 'struck', 'oz', 'brutality', 'unflinching', 'scene', 'violence', 'set', 'right', 'word', 'go', 'trust',
#  'show', 'faint', 'hearted', 'timid', 'show', 'pull', 'punch', 'regard', 'drug', 'sex', 'violence', 'hardcore', 'classic',
#  'use', 'word', 'br', 'br', 'called', 'oz', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary',
#  'focus', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cell', 'glass', 'front', 'face', 'inwards',
#  'privacy', 'high', 'agenda', 'em', 'city', 'home', 'many', 'aryan', 'muslim', 'gangsta', 'latino', 'christian', 'italian',
#  'irish', 'scuffle', 'death', 'stare', 'dodgy', 'dealing', 'shady', 'agreement', 'never', 'far', 'away', 'br', 'br',
#  'would', 'say', 'main', 'appeal', 'show', 'due', 'fact', 'go', 'show', 'dare', 'forget', 'pretty', 'picture', 'painted',
#  'mainstream', 'audience', 'forget', 'charm', 'forget', 'romance', 'oz', 'mess', 'around', 'first', 'episode', 'ever',
#  'saw', 'struck', 'nasty', 'surreal', 'say', 'ready', 'watched', 'developed', 'taste', 'oz', 'got', 'accustomed', 'high',
#  'level', 'graphic', 'violence', 'violence', 'injustice', 'crooked', 'guard', 'sold', 'nickel', 'inmate', 'kill', 'order',
#  'get', 'away', 'well', 'mannered', 'middle', 'class', 'inmate', 'turned', 'prison', 'bitch', 'due', 'lack', 'street',
#  'skill', 'prison', 'experience', 'watching', 'oz', 'may', 'become', 'comfortable', 'uncomfortable', 'viewing', 'thats',
#  'get', 'touch', 'darker', 'side', 'wonderful', 'little', 'production', 'br', 'br', 'filming', 'technique', 'unassuming',
#  'old', 'time', 'bbc', 'fashion', 'give', 'comforting', 'sometimes', 'discomforting', 'sense', 'realism', 'entire', 'piece',
#  'br', 'br', 'actor', 'extremely', 'well', 'chosen', 'michael', 'sheen', 'got', 'polari', 'voice', 'pat', 'truly', 'see',
#  'seamless', 'editing', 'guided', 'reference', 'williams', 'diary', 'entry', 'well', 'worth', 'watching', 'terrificly',
#  'written', 'performed', 'piece', 'masterful', 'production', 'one', 'great', 'master', 'comedy', 'life', 'br', 'br',
#  'realism', 'really', 'come', 'home', 'little', 'thing', 'fantasy', 'guard', 'rather', 'use', 'traditional', 'dream',
#  'technique', 'remains', 'solid', 'disappears', 'play', 'knowledge', 'sens', 'particularly', 'scene', 'concerning',
#  'orton', 'halliwell', 'set', 'particularly', 'flat', 'halliwell', 'mural', 'decorating', 'every', 'surface', 'terribly',
#  'well', 'done']
# 260

set used (i.e) Vocalbulary of data

In [ ]:
# ['br', 'technique', 'diary', 'get', 'romance', 'fact', 'regard', 'wonderful', 'crooked', 'given', 'viewing', 'pat',
#  'unflinching', 'city', 'comforting', 'entire', 'surface', 'mainstream', 'thing', 'face', 'italian', 'pretty', 'picture',
#  'oswald', 'irish', 'shady', 'punch', 'ever', 'glass', 'never', 'class', 'reference', 'drug', 'old', 'pull', 'main',
#  'ready', 'aryan', 'front', 'sens', 'written', 'polari', 'muslim', 'comedy', 'may', 'brutality', 'high', 'see', 'mainly',
#  'struck', 'agreement', 'scuffle', 'taste', 'sense', 'mannered', 'every', 'many', 'knowledge', 'word', 'scene', 'prison',
#  'set', 'happened', 'use', 'comfortable', 'accustomed', 'thats', 'actor', 'voice', 'focus', 'life', 'say', 'reviewer',
#  'watched', 'performed', 'give', 'agenda', 'violence', 'hooked', 'security', 'turned', 'saw', 'extremely', 'remains',
#  'latino', 'terribly', 'touch', 'dream', 'watching', 'street', 'experience', 'traditional', 'level', 'filming', 'side',
#  'around', 'injustice', 'piece', 'mentioned', 'home', 'forget', 'developed', 'time', 'flat', 'right', 'realism', 'seamless',
#  'experimental', 'fantasy', 'dodgy', 'one', 'decorating', 'little', 'play', 'episode', 'gangsta', 'faint', 'cell',
#  'williams', 'exactly', 'far', 'chosen', 'nickel', 'graphic', 'mural', 'oz', 'master', 'really', 'away', 'order',
#  'nasty', 'maximum', 'painted', 'terrificly', 'dealing', 'nickname', 'discomforting', 'em', 'darker', 'would', 'sex',
#  'section', 'editing', 'privacy', 'orton', 'bbc', 'masterful', 'appeal', 'trust', 'bitch', 'become', 'truly', 'classic',
#  'surreal', 'audience', 'done', 'great', 'disappears', 'solid', 'guided', 'uncomfortable', 'michael', 'production', 'sold',
#  'come', 'sometimes', 'concerning', 'entry', 'well', 'hardcore', 'got', 'fashion', 'mess', 'skill', 'particularly',
#  'middle', 'dare', 'worth', 'state', 'called', 'christian', 'due', 'charm', 'unassuming', 'go', 'death', 'halliwell',
#  'emerald', 'lack', 'penitentary', 'hearted', 'inwards', 'first', 'rather', 'inmate', 'kill', 'show', 'sheen', 'guard',
#  'stare', 'timid']
# 201

# Vector Conversion

In [ ]:

word2vec = Word2Vec(sentences = tokenized_words, vector_size=10, window=5, min_count=1, workers=4)

In [ ]:
word_vectors = {}
for corpus_wrods in tokenized_words:
    word_vectors[word] = word2vec.wv[word]

In [ ]:
# (word_vectors)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize an empty list to store word vectors for each tokenized sentence
X = []

# Iterate through each tokenized sentence
for sentence in tokenized_words:
    # Initialize an empty list to store word vectors for this sentence
    sentence_vectors = []
    # Iterate through each token in the sentence
    for token in sentence:
        # Check if the token exists in the vocabulary of the Word2Vec model
        if token in word_vectors:
            # Retrieve the word vector for this token from the Word2Vec model
            word_vector = word_vectors[token]
            # Append the word vector to the list of word vectors for this sentence
            sentence_vectors.append(word_vector)
    # Append the list of word vectors for this sentence to the list of sequences
    X.append(sentence_vectors)

# Pad the sequences to ensure they all have the same length
X_padded = pad_sequences(X, dtype='float32', padding='post')  # You can adjust dtype and padding according to your needs

# Convert the padded sequences to a numpy array
X_np = np.array(X_padded)


In [ ]:
#labels

In [ ]:

Y = np.array(labels)
X_np.shape, Y.shape

((50000, 300, 10), (50000,))

# Model Building

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

# Define maximum sequence length
max_len = 100
num_words = 50000
embedding_dim = 10


# Define your LSTM model architecture
model = Sequential()
model.add(LSTM(units = 100, input_shape(X.shape[1:]), activation = "tanh", return_sequence = True))
model.add(LSTM(units=100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())

# Padding sequences
# X_pad = pad_sequences(list(word_vectors.values()), maxlen=max_len)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_np, Y, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 10)           500000    
                                                                 
 spatial_dropout1d_3 (Spati  (None, 100, 10)           0         
 alDropout1D)                                                    
                                                                 
 lstm_3 (LSTM)               (None, 100)               44400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 544501 (2.08 MB)
Trainable params: 544501 (2.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/5


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 235, in assert_input_compatibility
        raise ValueError(

    ValueError: Ex